# XGBoost Parameter Tuning for Rent Listing Inqueries

Rental Listing Inquiries数据集是Kaggle平台上的一个分类竞赛任务，需要根据公寓的特征来预测其受欢迎程度（用户感兴趣程度分为高、中、低三类）。其中房屋的特征x共有14维，响应值y为用户对该公寓的感兴趣程度。评价标准为logloss。 数据链接：https://www.kaggle.com/c/two-sigma-connect-rental-listing-inquiries

# 第4步：调整正则化参数：reg_alpha

行列采样参数略

首先 import 必要的模块

In [1]:
from xgboost import XGBClassifier

import pandas as pd 
import numpy as np

import math

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import log_loss

from matplotlib import pyplot
%matplotlib inline

## 读取数据

In [2]:
# path to where the data lies
#dpath = './data/'
train = pd.read_csv("RentListingInquries_FE_train.csv")
#train.head()

y_train = train['interest_level']

train = train.drop([ "interest_level"], axis=1)
X_train = train

此前已经调好的参数：
n_estimators：232
max_depth：6
min_child_weight：7

用交叉验证评价模型性能时，用scoring参数定义评价指标。评价指标是越高越好，因此用一些损失函数当评价指标时，需要再加负号，如neg_log_loss，neg_mean_squared_error 详见sklearn文档：http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss

In [3]:
reg_alpha = [ 0.1, 1, 2]    #default = 0, 测试0.1,1，1.5，2
#reg_lambda = [0.5, 1, 2]      #default = 1，测试0.1， 0.5， 1，2

param_test4 = dict(reg_alpha=reg_alpha)
param_test4

{'reg_alpha': [0.1, 1, 2]}

In [4]:
xgb4 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=232,  #第二轮参数调整得到的n_estimators最优值
        max_depth=6,
        min_child_weight=7,
        gamma=0,
        subsample=0.5,
        colsample_bytree=0.8,
        colsample_bylevel = 0.7,
        objective= 'multi:softprob',
        seed=3)


gsearch4 = GridSearchCV(xgb4, param_grid = param_test4, scoring='neg_log_loss',n_jobs=-1, cv=3)
gsearch4.fit(X_train , y_train)

gsearch4.grid_scores_, gsearch4.best_params_,     gsearch4.best_score_

/Users/qing/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.57794, std: 0.00412, params: {'reg_alpha': 0.1},
  mean: -0.57819, std: 0.00488, params: {'reg_alpha': 1},
  mean: -0.57828, std: 0.00481, params: {'reg_alpha': 2}],
 {'reg_alpha': 0.1},
 -0.57794233118753435)

当reg_alpha取缺省值（0）时，logsloss=0.5777316363039906
比此时最佳的参数（0.1）还要低，以因此最佳的reg_alpha=0.